In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bvh_converter import bvh_mod
from scipy.signal import savgol_filter

from matplotlib.lines import Line2D
from scipy.interpolate import interp1d

## v1

In [4]:
def plot_foot_trajectories_by_subdiv(
    file_name: str,
    mode: str,
    subdiv_set: list,  # e.g. [2,5,8,11] or [3,6,9,12]
    base_path_cycles: str = "data/virtual_cycles",
    base_path_logs: str = "data/logs_v1_may",
    frame_rate: float = 240,
    time_segments: list = None,  # List of (start, end) tuples
    n_beats_per_cycle: int = 4,
    n_subdiv_per_beat: int = 12,
    nn: int = 8,
    figsize: tuple = (12, 6),
    dpi: int = 200,
    use_cycles: bool = True,
    show_gray_plots: bool = True
):
    """
    Plot left- and right-foot Y-position trajectories ±window around specified subdivisions,
    marking foot-onset times for cycles that have an onset in the window.
    Optionally plots trajectories for cycles without onsets in gray.

    Parameters
    ----------
    [previous parameters remain the same]
    time_segments : list of tuples
        List of (start, end) time segments to plot. If None, uses default window.
    """
    if len(subdiv_set) != n_beats_per_cycle:
        raise ValueError(f"subdiv_set must have length {n_beats_per_cycle}")

    # Use default window if no segments provided
    if time_segments is None:
        time_segments = [(170.0, 185.0)]

    # build file paths
    cycles_csv = os.path.join(base_path_cycles, f"{file_name}_C.csv")
    logs_onset_dir = os.path.join(base_path_logs, f"{file_name}_T", "onset_info")
    left_onsets_csv  = os.path.join(logs_onset_dir, f"{file_name}_T_left_foot_onsets.csv")
    right_onsets_csv = os.path.join(logs_onset_dir, f"{file_name}_T_right_foot_onsets.csv")
    left_zpos_csv    = os.path.join(logs_onset_dir, f"{file_name}_T_left_foot_zpos.csv")
    right_zpos_csv   = os.path.join(logs_onset_dir, f"{file_name}_T_right_foot_zpos.csv")

    # load data
    Lz = pd.read_csv(left_zpos_csv)["zpos"].values
    Rz = pd.read_csv(right_zpos_csv)["zpos"].values
    n_frames = len(Lz)
    times = np.arange(n_frames) / frame_rate

    # interpolation functions
    L_interp = interp1d(times, Lz, bounds_error=False, fill_value="extrapolate")
    R_interp = interp1d(times, Rz, bounds_error=False, fill_value="extrapolate")

    # Get overall time range for color mapping
    total_start = min(seg[0] for seg in time_segments)
    total_end = max(seg[1] for seg in time_segments)
    t_range = total_end - total_start

    # Calculate average cycle duration from all segments
    all_onsets = []
    for seg_start, seg_end in time_segments:
        cyc_df = pd.read_csv(cycles_csv)
        cyc_df = cyc_df[(cyc_df["Virtual Onset"] >= seg_start) & (cyc_df["Virtual Onset"] <= seg_end)]
        if not cyc_df.empty:
            all_onsets.extend(cyc_df["Virtual Onset"].values[:-1])
    
    if not all_onsets:
        raise ValueError("No cycles found in any of the time segments")
    
    # Calculate average cycle duration
    durations = np.diff(sorted(all_onsets))
    avg_cycle = durations.mean()

    # Calculate beat and subdivision lengths
    beat_len = avg_cycle / n_beats_per_cycle
    subdiv_len = beat_len / n_subdiv_per_beat
    half_win = subdiv_len * nn

    # Create figure with subplots for each beat
    fig, axes = plt.subplots(2, 2, figsize=figsize, dpi=dpi)
    axes = axes.flatten()
    cmap = plt.get_cmap('cool')

    # For each beat position (1,2,3,4)
    for beat_idx, ax in enumerate(axes):
        # Calculate time offset for this subdivision
        subdiv_offset = (subdiv_set[beat_idx] - 1) * subdiv_len  # -1 because subdivisions are 1-based
        beat_offset = beat_idx * beat_len
        total_offset = beat_offset + subdiv_offset

        # Process each time segment
        for seg_start, seg_end in time_segments:
            # trim to window
            win_mask = (times >= seg_start) & (times <= seg_end)
            t_win = times[win_mask]
            L_win = Lz[win_mask]
            R_win = Rz[win_mask]

            # cycles (downbeats)
            cyc_df = pd.read_csv(cycles_csv)
            cyc_df = cyc_df[(cyc_df["Virtual Onset"] >= seg_start) & (cyc_df["Virtual Onset"] <= seg_end)]
            onsets = cyc_df["Virtual Onset"].values[:-1]

            # foot onsets
            left_df  = pd.read_csv(left_onsets_csv)
            right_df = pd.read_csv(right_onsets_csv)
            left_times  = left_df[ (left_df["time_sec"]>=seg_start)&(left_df["time_sec"]<=seg_end) ]["time_sec"].values
            right_times = right_df[(right_df["time_sec"]>=seg_start)&(right_df["time_sec"]<=seg_end)]["time_sec"].values

            # Plot gray trajectories if enabled
            if show_gray_plots:
                for c in onsets:
                    subdiv_time = c + total_offset
                    m = (t_win>=subdiv_time-half_win)&(t_win<=subdiv_time+half_win)
                    tr = t_win[m] - subdiv_time
                    if use_cycles:
                        tr = tr / avg_cycle
                    ax.plot(tr, L_win[m], '-', color='gray', alpha=0.1)
                    ax.plot(tr, R_win[m], '--', color='gray', alpha=0.1)

            # Collect cycles that have foot onsets near this subdivision
            cyc_L, L_near = [], {}
            cyc_R, R_near = [], {}
            
            for c in onsets:
                subdiv_time = c + total_offset
                # Left foot
                hits = left_times[(left_times>=subdiv_time-half_win)&(left_times<=subdiv_time+half_win)]
                if len(hits):
                    cyc_L.append(c)
                    L_near[c] = hits
                # Right foot
                hits = right_times[(right_times>=subdiv_time-half_win)&(right_times<=subdiv_time+half_win)]
                if len(hits):
                    cyc_R.append(c)
                    R_near[c] = hits

            # Plot left foot trajectories with onsets
            for i, c in enumerate(cyc_L):
                col = cmap((c-total_start)/t_range)
                subdiv_time = c + total_offset
                m = (t_win>=subdiv_time-half_win)&(t_win<=subdiv_time+half_win)
                tr = t_win[m] - subdiv_time
                if use_cycles:
                    tr = tr / avg_cycle
                ax.plot(tr, L_win[m], '-', color=col, alpha=0.3,
                        label="Left Foot" if i==0 else "")
                for lt in L_near[c]:
                    rel = lt - subdiv_time
                    if use_cycles:
                        rel = rel / avg_cycle
                    ax.axvline(rel, color=col, linestyle='-', alpha=0.5)
                    ax.plot(rel, L_interp(lt), 'o', ms=8, markeredgecolor='k', alpha=0.8)

            # Plot right foot trajectories with onsets
            for i, c in enumerate(cyc_R):
                col = cmap((c-total_start)/t_range)
                subdiv_time = c + total_offset
                m = (t_win>=subdiv_time-half_win)&(t_win<=subdiv_time+half_win)
                tr = t_win[m] - subdiv_time
                if use_cycles:
                    tr = tr / avg_cycle
                ax.plot(tr, R_win[m], '--', color=col, alpha=0.3,
                        label="Right Foot" if i==0 else "")
                for rt in R_near[c]:
                    rel = rt - subdiv_time
                    if use_cycles:
                        rel = rel / avg_cycle
                    ax.axvline(rel, color=col, linestyle='--', alpha=0.5)
                    ax.plot(rel, R_interp(rt), 'x', ms=8, markeredgecolor='k', alpha=0.8)

        # Decorations for each subplot
        ax.axvline(0, color='k', linewidth=1.5, label="Subdivision (t=0)")
        for j in range(-nn, nn+1):
            if j!=0:
                pos = j*subdiv_len
                if use_cycles:
                    pos = pos / avg_cycle
                ax.axvline(pos, color='gray', linestyle=':', alpha=0.5)

        xlabel = "Cycles relative to subdivision" if use_cycles else "Time relative to subdivision (s)"
        ax.set_xlabel(xlabel)
        ax.set_ylabel("Foot Y Position")
        ax.set_title(f"Beat {beat_idx + 1}, Subdivision {subdiv_set[beat_idx]}")
        ax.grid(True, alpha=0.3)

        # Add legend only to first subplot
        if beat_idx == 0:
            custom = [
                Line2D([0],[0],color='blue',linestyle='-', lw=2),
                Line2D([0],[0],marker='o', color='w', markerfacecolor='blue', ms=8, markeredgecolor='k'),
                Line2D([0],[0],color='blue',linestyle='--', lw=2),
                Line2D([0],[0],marker='x', color='w', markeredgecolor='blue', ms=8),
                Line2D([0],[0],color='k', lw=2)
            ]
            labels = ["Left Trajectory","Left Onset","Right Trajectory",
                     "Right Onset","Subdivision (t=0)"]
            ax.legend(custom, labels, loc='upper left', framealpha=0.3)

    # Add colorbar to the figure
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(total_start, total_end))
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=axes.ravel().tolist())
    cbar.set_label('Time in recording (s)')

    # Create segment labels for title
    segment_labels = [f"{start:.1f}-{end:.1f}s" for start, end in time_segments]
    segment_str = " | ".join(segment_labels)

    plt.suptitle(
        f"Foot Trajectories ±{2*nn/n_subdiv_per_beat/ n_beats_per_cycle:.1f} beats around subdivisions {subdiv_set}\n"
        f"{file_name} | segments: {segment_str} | {mode}",
        fontsize=10
    )
    
    plt.subplots_adjust(top=0.85)
    fig.set_constrained_layout(True)

    return fig, axes

In [ ]:
mode_csv_list = os.listdir("data/subset_dance_annotation")

for mode_csv in mode_csv_list:
    file_name = mode_csv.split("_Dancers")[0]
    mode_df = pd.read_csv("data/subset_dance_annotation/" + mode_csv)

    mode_group = mode_df[mode_df["mocap"] == "gr"].reset_index(drop=True)
    mode_individual = mode_df[mode_df["mocap"] == "in"].reset_index(drop=True)
    mode_audience = mode_df[mode_df["mocap"] == "au"].reset_index(drop=True)

    # helper to extract all (start, end) tuples for a mode
    def get_segments(df, name):
        if df.empty:
            print(f"⚠️  No rows for mode '{name}', skipping.")
            return None
        return [(row["Start (in sec)"], row["End (in sec)"]) for _, row in df.iterrows()]

    # build a dict of segments
    segments = {
        "group":      get_segments(mode_group,      "gr"),
        "individual": get_segments(mode_individual, "in"),
        "audience":   get_segments(mode_audience,   "au")
    }

    # filter out the empty ones
    tsegment = {mode: seg for mode, seg in segments.items() if seg is not None}

    save_dir = f"output_static_plot/foot_trajectories/{file_name}"
    os.makedirs(save_dir, exist_ok=True)
    for mode, segments in tsegment.items():
        fig, ax = plot_foot_trajectories_by_subdiv(
            file_name=file_name,
            mode=mode,
            base_path_cycles="data/virtual_cycles",
            base_path_logs="data/logs_v1_may",
            time_segments=segments,  # Pass all segments for this mode
            n_beats_per_cycle=4, 
            n_subdiv_per_beat=12, 
            nn=8,
            use_cycles=True,
            show_gray_plots=True,
            subdiv_set=[2,5,8,11]
        )
        # Create a filename that includes all segments
        # segment_str = "_".join([f"{start:.1f}_{end:.1f}" for start, end in segments])
        fig.savefig(os.path.join(save_dir, f"{file_name}_{mode}.png"))
        plt.close(fig)

## v2: plot_foot_trajectories_by_subdiv multi mode

In [2]:
def plot_foot_trajectories_by_subdiv(
    file_name: str,
    mode: str,
    subdiv_set: list,  # e.g. [2,5,8,11] or [3,6,9,12]
    base_path_cycles: str = "data/virtual_cycles",
    base_path_logs: str = "data/logs_v1_may",
    frame_rate: float = 240,
    time_segments: list = None,  # List of (start, end) tuples
    n_beats_per_cycle: int = 4,
    n_subdiv_per_beat: int = 12,
    nn: int = 8,
    figsize: tuple = (12, 6),
    dpi: int = 200,
    use_cycles: bool = True,
    show_gray_plots: bool = True,
    show_trajectories: bool = True,  # New parameter to control trajectory lines
    show_vlines: bool = True        # New parameter to control vertical lines
):
    """
    Plot left- and right-foot Y-position trajectories ±window around specified subdivisions,
    marking foot-onset times for cycles that have an onset in the window.
    Optionally plots trajectories for cycles without onsets in gray.

    Parameters
    ----------
    [previous parameters remain the same]
    show_trajectories : bool
        Whether to show the continuous trajectory lines
    show_vlines : bool
        Whether to show vertical lines at onset times
    """
    if len(subdiv_set) != n_beats_per_cycle:
        raise ValueError(f"subdiv_set must have length {n_beats_per_cycle}")

    # Use default window if no segments provided
    if time_segments is None:
        time_segments = [(170.0, 185.0)]

    # build file paths
    cycles_csv = os.path.join(base_path_cycles, f"{file_name}_C.csv")
    logs_onset_dir = os.path.join(base_path_logs, f"{file_name}_T", "onset_info")
    left_onsets_csv  = os.path.join(logs_onset_dir, f"{file_name}_T_left_foot_onsets.csv")
    right_onsets_csv = os.path.join(logs_onset_dir, f"{file_name}_T_right_foot_onsets.csv")
    left_zpos_csv    = os.path.join(logs_onset_dir, f"{file_name}_T_left_foot_zpos.csv")
    right_zpos_csv   = os.path.join(logs_onset_dir, f"{file_name}_T_right_foot_zpos.csv")

    # load data
    Lz = pd.read_csv(left_zpos_csv)["zpos"].values
    Rz = pd.read_csv(right_zpos_csv)["zpos"].values
    n_frames = len(Lz)
    times = np.arange(n_frames) / frame_rate

    # interpolation functions
    L_interp = interp1d(times, Lz, bounds_error=False, fill_value="extrapolate")
    R_interp = interp1d(times, Rz, bounds_error=False, fill_value="extrapolate")

    # Get overall time range for color mapping
    total_start = min(seg[0] for seg in time_segments)
    total_end = max(seg[1] for seg in time_segments)
    t_range = total_end - total_start

    # Calculate average cycle duration from all segments
    all_onsets = []
    for seg_start, seg_end in time_segments:
        cyc_df = pd.read_csv(cycles_csv)
        cyc_df = cyc_df[(cyc_df["Virtual Onset"] >= seg_start) & (cyc_df["Virtual Onset"] <= seg_end)]
        if not cyc_df.empty:
            all_onsets.extend(cyc_df["Virtual Onset"].values[:-1])
    
    if not all_onsets:
        raise ValueError("No cycles found in any of the time segments")
    
    # Calculate average cycle duration
    durations = np.diff(sorted(all_onsets))
    avg_cycle = durations.mean()

    # Calculate beat and subdivision lengths
    beat_len = avg_cycle / n_beats_per_cycle
    subdiv_len = beat_len / n_subdiv_per_beat
    half_win = subdiv_len * nn

    # Create figure with subplots for each beat
    fig, axes = plt.subplots(2, 2, figsize=figsize, dpi=dpi)
    axes = axes.flatten()
    cmap = plt.get_cmap('cool')

    # For each beat position (1,2,3,4)
    for beat_idx, ax in enumerate(axes):
        # Calculate time offset for this subdivision
        subdiv_offset = (subdiv_set[beat_idx] - 1) * subdiv_len  # -1 because subdivisions are 1-based
        beat_offset = beat_idx * beat_len
        total_offset = beat_offset + subdiv_offset

        # Process each time segment
        for seg_start, seg_end in time_segments:
            # trim to window
            win_mask = (times >= seg_start) & (times <= seg_end)
            t_win = times[win_mask]
            L_win = Lz[win_mask]
            R_win = Rz[win_mask]

            # cycles (downbeats)
            cyc_df = pd.read_csv(cycles_csv)
            cyc_df = cyc_df[(cyc_df["Virtual Onset"] >= seg_start) & (cyc_df["Virtual Onset"] <= seg_end)]
            onsets = cyc_df["Virtual Onset"].values[:-1]

            # foot onsets
            left_df  = pd.read_csv(left_onsets_csv)
            right_df = pd.read_csv(right_onsets_csv)
            left_times  = left_df[ (left_df["time_sec"]>=seg_start)&(left_df["time_sec"]<=seg_end) ]["time_sec"].values
            right_times = right_df[(right_df["time_sec"]>=seg_start)&(right_df["time_sec"]<=seg_end)]["time_sec"].values

            # Plot gray trajectories if enabled
            if show_gray_plots and show_trajectories:
                for c in onsets:
                    subdiv_time = c + total_offset
                    m = (t_win>=subdiv_time-half_win)&(t_win<=subdiv_time+half_win)
                    tr = t_win[m] - subdiv_time
                    if use_cycles:
                        tr = tr / avg_cycle
                    ax.plot(tr, L_win[m], '-', color='gray', alpha=0.1)
                    ax.plot(tr, R_win[m], '--', color='gray', alpha=0.1)

            # Collect cycles that have foot onsets near this subdivision
            cyc_L, L_near = [], {}
            cyc_R, R_near = [], {}
            
            for c in onsets:
                subdiv_time = c + total_offset
                # Left foot
                hits = left_times[(left_times>=subdiv_time-half_win)&(left_times<=subdiv_time+half_win)]
                if len(hits):
                    cyc_L.append(c)
                    L_near[c] = hits
                # Right foot
                hits = right_times[(right_times>=subdiv_time-half_win)&(right_times<=subdiv_time+half_win)]
                if len(hits):
                    cyc_R.append(c)
                    R_near[c] = hits

            # Plot left foot trajectories with onsets
            for i, c in enumerate(cyc_L):
                col = cmap((c-total_start)/t_range)
                subdiv_time = c + total_offset
                m = (t_win>=subdiv_time-half_win)&(t_win<=subdiv_time+half_win)
                tr = t_win[m] - subdiv_time
                if use_cycles:
                    tr = tr / avg_cycle
                if show_trajectories:
                    ax.plot(tr, L_win[m], '-', color=col, alpha=0.3,
                            label="Left Foot" if i==0 else "")
                for lt in L_near[c]:
                    rel = lt - subdiv_time  # or beat_time for by_beat version
                    if use_cycles:
                        rel = rel / avg_cycle
                    if show_vlines:
                        ax.axvline(rel, color=col, linestyle='-', alpha=0.5)
                    # Change this line to use a fixed color for left foot markers
                    ax.plot(rel, L_interp(lt), 'o', ms=8, markeredgecolor='k', 
                            markerfacecolor='blue', alpha=0.8)  # Fixed blue color for left foot

            # Plot right foot trajectories with onsets
            for i, c in enumerate(cyc_R):
                col = cmap((c-total_start)/t_range)
                subdiv_time = c + total_offset
                m = (t_win>=subdiv_time-half_win)&(t_win<=subdiv_time+half_win)
                tr = t_win[m] - subdiv_time
                if use_cycles:
                    tr = tr / avg_cycle
                if show_trajectories:
                    ax.plot(tr, R_win[m], '--', color=col, alpha=0.3,
                            label="Right Foot" if i==0 else "")
                for rt in R_near[c]:
                    rel = rt - subdiv_time  # or beat_time for by_beat version
                    if use_cycles:
                        rel = rel / avg_cycle
                    if show_vlines:
                        ax.axvline(rel, color=col, linestyle='--', alpha=0.5)
                    # Change this line to use a fixed color for right foot markers
                    ax.plot(rel, R_interp(rt), 'x', ms=8, markeredgecolor='red', 
                            color='red', alpha=0.8)  # Fixed red color for right foot

        # Decorations for each subplot
        ax.axvline(0, color='k', linewidth=1.5, label="Subdivision (t=0)")
        for j in range(-nn, nn+1):
            if j!=0:
                pos = j*subdiv_len
                if use_cycles:
                    pos = pos / avg_cycle
                ax.axvline(pos, color='gray', linestyle=':', alpha=0.5)

        xlabel = "Cycles relative to subdivision" if use_cycles else "Time relative to subdivision (s)"
        ax.set_xlabel(xlabel)
        ax.set_ylabel("Foot Y Position")
        ax.set_title(f"Beat {beat_idx + 1}, Subdivision {subdiv_set[beat_idx]}")
        ax.grid(True, alpha=0.3)

        # Add legend only to first subplot
        if beat_idx == 0:
            custom = [
                # Line2D([0],[0],color='blue',linestyle='-', lw=2),
                Line2D([0],[0],marker='o', color='w', markerfacecolor='blue', ms=8, markeredgecolor='k'),
                # Line2D([0],[0],color='red',linestyle='--', lw=2),
                Line2D([0],[0],marker='x', color='red', ms=8),
                Line2D([0],[0],color='k', lw=2)
            ]
            # labels = ["Left Trajectory","Left Onset","Right Trajectory",
            #         "Right Onset","Beat (t=0)"] 
            labels = ["Left Onset", "Right Onset","Beat (t=0)"] 
            
            ax.legend(custom, labels, loc='upper left', framealpha=0.3)

    # Add colorbar to the figure
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(total_start, total_end))
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=axes.ravel().tolist())
    cbar.set_label('Time in recording (s)')

    # Create segment labels for title
    segment_labels = [f"{start:.1f}-{end:.1f}s" for start, end in time_segments]
    segment_str = " | ".join(segment_labels)

    plt.suptitle(
        f"Foot Trajectories ±{2*nn/n_subdiv_per_beat/ n_beats_per_cycle:.1f} beats around subdivisions {subdiv_set}\n"
        f"{file_name} | segments: all | {mode}",
        fontsize=10
    )
    
    plt.subplots_adjust(top=0.85)
    fig.set_constrained_layout(True)

    return fig, axes

In [3]:
mode_csv_list = os.listdir("data/subset_dance_annotation")

for mode_csv in mode_csv_list:
    file_name = mode_csv.split("_Dancers")[0]
    mode_df = pd.read_csv("data/subset_dance_annotation/" + mode_csv)

    mode_group = mode_df[mode_df["mocap"] == "gr"].reset_index(drop=True)
    mode_individual = mode_df[mode_df["mocap"] == "in"].reset_index(drop=True)
    mode_audience = mode_df[mode_df["mocap"] == "au"].reset_index(drop=True)

    # helper to extract all (start, end) tuples for a mode
    def get_segments(df, name):
        if df.empty:
            print(f"⚠️  No rows for mode '{name}', skipping.")
            return None
        return [(row["Start (in sec)"], row["End (in sec)"]) for _, row in df.iterrows()]

    # build a dict of segments
    segments = {
        "group":      get_segments(mode_group,      "gr"),
        "individual": get_segments(mode_individual, "in"),
        "audience":   get_segments(mode_audience,   "au")
    }

    # filter out the empty ones
    tsegment = {mode: seg for mode, seg in segments.items() if seg is not None}

    save_dir = f"output_static_plot/foot_trajectories/{file_name}"
    os.makedirs(save_dir, exist_ok=True)
    for mode, segments in tsegment.items():
        fig, ax = plot_foot_trajectories_by_subdiv(
            file_name=file_name,
            mode=mode,
            base_path_cycles="data/virtual_cycles",
            base_path_logs="data/logs_v1_may",
            time_segments=segments,  # Pass all segments for this mode
            n_beats_per_cycle=4, 
            n_subdiv_per_beat=12, 
            nn=8,
            use_cycles=True,
            show_gray_plots=False,
            subdiv_set=[2,5,8,11],
            show_trajectories=False,  # New parameter to control trajectory lines
            show_vlines=False        # New parameter to control vertical lines
        )
        # Create a filename that includes all segments
        # segment_str = "_".join([f"{start:.1f}_{end:.1f}" for start, end in segments])
        fig.savefig(os.path.join(save_dir, f"{file_name}_{mode}.png"))
        plt.close(fig)

⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.


C:\Users\Sagar\AppData\Local\Temp\ipykernel_7876\3337985891.py:75: RuntimeWarning: Mean of empty slice.
  avg_cycle = durations.mean()
c:\ProgramData\Anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'in', skipping.


## 

In [4]:
def plot_foot_trajectories_by_beat(
    file_name: str,
    mode: str,
    base_path_cycles: str = "data/virtual_cycles",
    base_path_logs: str = "data/logs_v1_may",
    frame_rate: float = 240,
    time_segments: list = None,  # List of (start, end) tuples
    n_beats_per_cycle: int = 4,
    n_subdiv_per_beat: int = 12,
    nn: int = 8,
    figsize: tuple = (12, 6),
    dpi: int = 200,
    use_cycles: bool = True,
    show_gray_plots: bool = True,
    show_trajectories: bool = True,  # New parameter to control trajectory lines
    show_vlines: bool = True        # New parameter to control vertical lines
):
    """
    Plot left- and right-foot Y-position trajectories ±window around each beat (1,2,3,4),
    marking foot-onset times for cycles that have an onset in the window.
    Optionally plots trajectories for cycles without onsets in gray.

    Parameters
    ----------
    [previous parameters remain the same]
    time_segments : list of tuples
        List of (start, end) time segments to plot. If None, uses default window.
    show_trajectories : bool
        Whether to show the continuous trajectory lines
    show_vlines : bool
        Whether to show vertical lines at onset times
    """
    # Use default window if no segments provided
    if time_segments is None:
        time_segments = [(170.0, 185.0)]

    # build file paths
    cycles_csv = os.path.join(base_path_cycles, f"{file_name}_C.csv")
    logs_onset_dir = os.path.join(base_path_logs, f"{file_name}_T", "onset_info")
    left_onsets_csv  = os.path.join(logs_onset_dir, f"{file_name}_T_left_foot_onsets.csv")
    right_onsets_csv = os.path.join(logs_onset_dir, f"{file_name}_T_right_foot_onsets.csv")
    left_zpos_csv    = os.path.join(logs_onset_dir, f"{file_name}_T_left_foot_zpos.csv")
    right_zpos_csv   = os.path.join(logs_onset_dir, f"{file_name}_T_right_foot_zpos.csv")

    # load data
    Lz = pd.read_csv(left_zpos_csv)["zpos"].values
    Rz = pd.read_csv(right_zpos_csv)["zpos"].values
    n_frames = len(Lz)
    times = np.arange(n_frames) / frame_rate

    # interpolation functions
    L_interp = interp1d(times, Lz, bounds_error=False, fill_value="extrapolate")
    R_interp = interp1d(times, Rz, bounds_error=False, fill_value="extrapolate")

    # Get overall time range for color mapping
    total_start = min(seg[0] for seg in time_segments)
    total_end = max(seg[1] for seg in time_segments)
    t_range = total_end - total_start

    # Calculate average cycle duration from all segments
    all_onsets = []
    for seg_start, seg_end in time_segments:
        cyc_df = pd.read_csv(cycles_csv)
        cyc_df = cyc_df[(cyc_df["Virtual Onset"] >= seg_start) & (cyc_df["Virtual Onset"] <= seg_end)]
        if not cyc_df.empty:
            all_onsets.extend(cyc_df["Virtual Onset"].values[:-1])
    
    if not all_onsets:
        raise ValueError("No cycles found in any of the time segments")
    
    # Calculate average cycle duration
    durations = np.diff(sorted(all_onsets))
    avg_cycle = durations.mean()

    # Calculate beat and subdivision lengths
    beat_len = avg_cycle / n_beats_per_cycle
    subdiv_len = beat_len / n_subdiv_per_beat
    half_win = subdiv_len * nn

    # Create figure with subplots for each beat
    fig, axes = plt.subplots(2, 2, figsize=figsize, dpi=dpi)
    axes = axes.flatten()
    cmap = plt.get_cmap('cool')

    # For each beat position (1,2,3,4)
    for beat_idx, ax in enumerate(axes):
        beat_offset = beat_idx * beat_len  # Time offset for this beat
        
        # Process each time segment
        for seg_start, seg_end in time_segments:
            # trim to window
            win_mask = (times >= seg_start) & (times <= seg_end)
            t_win = times[win_mask]
            L_win = Lz[win_mask]
            R_win = Rz[win_mask]

            # cycles (downbeats)
            cyc_df = pd.read_csv(cycles_csv)
            cyc_df = cyc_df[(cyc_df["Virtual Onset"] >= seg_start) & (cyc_df["Virtual Onset"] <= seg_end)]
            onsets = cyc_df["Virtual Onset"].values[:-1]

            # foot onsets
            left_df  = pd.read_csv(left_onsets_csv)
            right_df = pd.read_csv(right_onsets_csv)
            left_times  = left_df[ (left_df["time_sec"]>=seg_start)&(left_df["time_sec"]<=seg_end) ]["time_sec"].values
            right_times = right_df[(right_df["time_sec"]>=seg_start)&(right_df["time_sec"]<=seg_end)]["time_sec"].values

            # Plot gray trajectories if enabled
            if show_gray_plots and show_trajectories:
                for c in onsets:
                    beat_time = c + beat_offset  # Time of this beat in this cycle
                    m = (t_win>=beat_time-half_win)&(t_win<=beat_time+half_win)
                    tr = t_win[m] - beat_time
                    if use_cycles:
                        tr = tr / avg_cycle
                    ax.plot(tr, L_win[m], '-', color='gray', alpha=0.1)
                    ax.plot(tr, R_win[m], '--', color='gray', alpha=0.1)

            # Collect cycles that have foot onsets near this beat
            cyc_L, L_near = [], {}
            cyc_R, R_near = [], {}
            
            for c in onsets:
                beat_time = c + beat_offset
                # Left foot
                hits = left_times[(left_times>=beat_time-half_win)&(left_times<=beat_time+half_win)]
                if len(hits):
                    cyc_L.append(c)
                    L_near[c] = hits
                # Right foot
                hits = right_times[(right_times>=beat_time-half_win)&(right_times<=beat_time+half_win)]
                if len(hits):
                    cyc_R.append(c)
                    R_near[c] = hits

            # Plot left foot trajectories with onsets
            for i, c in enumerate(cyc_L):
                col = cmap((c-total_start)/t_range)
                beat_time = c + beat_offset
                m = (t_win>=beat_time-half_win)&(t_win<=beat_time+half_win)
                tr = t_win[m] - beat_time
                if use_cycles:
                    tr = tr / avg_cycle
                if show_trajectories:
                    ax.plot(tr, L_win[m], '-', color=col, alpha=0.3,
                            label="Left Foot" if i==0 else "")
                for lt in L_near[c]:
                    rel = lt - beat_time  # or beat_time for by_beat version
                    if use_cycles:
                        rel = rel / avg_cycle
                    if show_vlines:
                        ax.axvline(rel, color=col, linestyle='-', alpha=0.5)
                    # Change this line to use a fixed color for left foot markers
                    ax.plot(rel, L_interp(lt), 'o', ms=8, markeredgecolor='k', 
                            markerfacecolor='blue', alpha=0.8)  # Fixed blue color for left foot

            # Plot right foot trajectories with onsets
            for i, c in enumerate(cyc_R):
                col = cmap((c-total_start)/t_range)
                beat_time = c + beat_offset
                m = (t_win>=beat_time-half_win)&(t_win<=beat_time+half_win)
                tr = t_win[m] - beat_time
                if use_cycles:
                    tr = tr / avg_cycle
                if show_trajectories:
                    ax.plot(tr, R_win[m], '--', color=col, alpha=0.3,
                            label="Right Foot" if i==0 else "")
                for rt in R_near[c]:
                    rel = rt - beat_time  # or beat_time for by_beat version
                    if use_cycles:
                        rel = rel / avg_cycle
                    if show_vlines:
                        ax.axvline(rel, color=col, linestyle='--', alpha=0.5)
                    # Change this line to use a fixed color for right foot markers
                    ax.plot(rel, R_interp(rt), 'x', ms=8, markeredgecolor='red', 
                            color='red', alpha=0.8)  # Fixed red color for right foot

        # Decorations for each subplot
        ax.axvline(0, color='k', linewidth=1.5, label="Beat (t=0)")
        for j in range(-nn, nn+1):
            if j!=0:
                pos = j*subdiv_len
                if use_cycles:
                    pos = pos / avg_cycle
                ax.axvline(pos, color='gray', linestyle=':', alpha=0.5)

        xlabel = "Cycles relative to beat" if use_cycles else "Time relative to beat (s)"
        ax.set_xlabel(xlabel)
        ax.set_ylabel("Foot Y Position")
        ax.set_title(f"Beat {beat_idx + 1}")
        ax.grid(True, alpha=0.3)

        # Add legend only to first subplot
        if beat_idx == 0:
            custom = [
                # Line2D([0],[0],color='blue',linestyle='-', lw=2),
                Line2D([0],[0],marker='o', color='w', markerfacecolor='blue', ms=8, markeredgecolor='k'),
                # Line2D([0],[0],color='red',linestyle='--', lw=2),
                Line2D([0],[0],marker='x', color='red', ms=8),
                Line2D([0],[0],color='k', lw=2)
            ]
            # labels = ["Left Trajectory","Left Onset","Right Trajectory",
            #         "Right Onset","Beat (t=0)"] 
            labels = ["Left Onset", "Right Onset","Beat (t=0)"] 
             
            ax.legend(custom, labels, loc='upper left', framealpha=0.3)

    # Add colorbar to the figure
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(total_start, total_end))
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=axes.ravel().tolist())
    cbar.set_label('Time in recording (s)')

    # Create segment labels for title
    segment_labels = [f"{start:.1f}-{end:.1f}s" for start, end in time_segments]
    segment_str = " | ".join(segment_labels)

    plt.suptitle(
        f"Foot Trajectories ±{2*nn/n_subdiv_per_beat/ n_beats_per_cycle:.1f} beats around each beat\n"
        f"{file_name} | segments: all | {mode}",
        fontsize=10
    )
    
    plt.subplots_adjust(top=0.85)
    fig.set_constrained_layout(True)

    return fig, axes

In [ ]:
mode_csv_list = os.listdir("data/subset_dance_annotation")

for mode_csv in mode_csv_list:
    file_name = mode_csv.split("_Dancers")[0]
    mode_df = pd.read_csv("data/subset_dance_annotation/" + mode_csv)

    mode_group = mode_df[mode_df["mocap"] == "gr"].reset_index(drop=True)
    mode_individual = mode_df[mode_df["mocap"] == "in"].reset_index(drop=True)
    mode_audience = mode_df[mode_df["mocap"] == "au"].reset_index(drop=True)

    # helper to extract all (start, end) tuples for a mode
    def get_segments(df, name):
        if df.empty:
            print(f"⚠️  No rows for mode '{name}', skipping.")
            return None
        return [(row["Start (in sec)"], row["End (in sec)"]) for _, row in df.iterrows()]

    # build a dict of segments
    segments = {
        "group":      get_segments(mode_group,      "gr"),
        "individual": get_segments(mode_individual, "in"),
        "audience":   get_segments(mode_audience,   "au")
    }

    # filter out the empty ones
    tsegment = {mode: seg for mode, seg in segments.items() if seg is not None}

    save_dir = f"output_static_plot/foot_trajectories/{file_name}"
    os.makedirs(save_dir, exist_ok=True)
    for mode, segments in tsegment.items():
        fig, ax = plot_foot_trajectories_by_beat(
            file_name=file_name,
            mode=mode,
            base_path_cycles="data/virtual_cycles",
            base_path_logs="data/logs_v1_may",
            time_segments=segments,  # Pass all segments for this mode
            n_beats_per_cycle=4, 
            n_subdiv_per_beat=12, 
            nn=8,
            use_cycles=True,
            show_gray_plots=False,
            show_trajectories=False,  # New parameter to control trajectory lines
            show_vlines=False        # New parameter to control vertical lines
        )
        # Create a filename that includes all segments
        # segment_str = "_".join([f"{start:.1f}_{end:.1f}" for start, end in segments])
        fig.savefig(os.path.join(save_dir, f"{file_name}_{mode}.png"))
        plt.close(fig)

⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.


C:\Users\Sagar\AppData\Local\Temp\ipykernel_7876\3290379724.py:73: RuntimeWarning: Mean of empty slice.
  avg_cycle = durations.mean()
c:\ProgramData\Anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
